# <font color='blue'>Data Science Academy</font>
# <font color='blue'>Deep Learning Para Aplicações de IA com PyTorch e Lightning</font>

## <font color='blue'>Estudo de Caso 3</font>
## <font color='blue'>Usando Transformers Para Detecção de Anomalias em Texto</font>

![DSA](imagens/EC3.png)

## Instalando e Carregando os Pacotes

In [1]:
# Versão da Linguagem Python
from platform import python_version
print('Versão da Linguagem Python Usada Neste Jupyter Notebook:', python_version())

Versão da Linguagem Python Usada Neste Jupyter Notebook: 3.9.13


In [2]:
# Para atualizar um pacote, execute o comando abaixo no terminal ou prompt de comando:
# pip install -U nome_pacote

# Para instalar a versão exata de um pacote, execute o comando abaixo no terminal ou prompt de comando:
# !pip install nome_pacote==versão_desejada

# Depois de instalar ou atualizar o pacote, reinicie o jupyter notebook.

# Instala o pacote watermark. 
# Esse pacote é usado para gravar as versões de outros pacotes usados neste jupyter notebook.
#!pip install -q -U watermark

In [3]:
!pip install -q torch==2.0.0

In [4]:
!pip install -q transformers==4.28.1

In [5]:
# Imports
import torch
import sklearn
import numpy as np
from transformers import BertTokenizer, BertForSequenceClassification
from torch.utils.data import DataLoader, Dataset
from sklearn.model_selection import train_test_split

In [6]:
# Mostra somente mensagens de erro
from transformers import logging
logging.set_verbosity_error()

In [7]:
# Versões dos pacotes usados neste jupyter notebook
%reload_ext watermark
%watermark -a "Data Science Academy" --iversions

Author: Data Science Academy

numpy       : 1.21.5
sklearn     : 1.0.2
torch       : 2.0.0
transformers: 4.28.1



## Construção da Classe de Tokenização dos Dados

In [8]:
# Classe para tokenização dos dados
class TokenizaDados(Dataset):
    
    # Método construtor
    def __init__(self, texts, labels, tokenizer, max_length):
        self.texts = texts
        self.labels = labels
        self.tokenizer = tokenizer
        self.max_length = max_length

    # Método para calcular o comprimento do texto (cada sentença)
    def __len__(self):
        return len(self.texts)

    # Método para obter um item tokenizado
    def __getitem__(self, idx):
        
        # Obtém o índice do texto e do label
        text = self.texts[idx]
        label = self.labels[idx]
        
        # Aplica a tokenização
        inputs = self.tokenizer.encode_plus(text,
                                            add_special_tokens = True,
                                            max_length = self.max_length,
                                            padding = 'max_length',
                                            truncation = True,
                                            return_tensors = 'pt')

        return {
            'input_ids': inputs['input_ids'].squeeze(0),
            'attention_mask': inputs['attention_mask'].squeeze(0),
            'label': torch.tensor(label)
        }

## Funções Para os Loops de Treino, Avaliação e Previsão com Novos Dados

In [9]:
# Método do loop de treino
def treina_modelo(model, data_loader, criterion, optimizer, device):
    
    # Coloca o modelo em modo de treino
    model.train()
    
    # Inicializa o erro com zero
    total_loss = 0

    # Loop pelo data loader
    for batch in data_loader:
        
        # Extrai os ids do batch de dados e coloca no device
        input_ids = batch['input_ids'].to(device)
        
        # Extrai a máscara e coloca no device
        attention_mask = batch['attention_mask'].to(device)
        
        # Extrai os labels e coloca no device
        labels = batch['label'].to(device)

        # Zera os gradientes
        optimizer.zero_grad()
        
        # Faz as previsões
        outputs = model(input_ids, attention_mask = attention_mask, labels = labels)
        
        # Extrai o erro do modelo
        loss = outputs.loss
        
        # Aplica a otimização com backpropagation
        loss.backward()
        optimizer.step()

        # Acumula o erro
        total_loss += loss.item()

    return total_loss / len(data_loader)

In [10]:
# Método do loop de avaliação
def avalia_modelo(model, data_loader, criterion, device):
    
    model.eval()
    
    total_loss = 0

    with torch.no_grad():
    
        for batch in data_loader:
            
            input_ids = batch['input_ids'].to(device)
            
            attention_mask = batch['attention_mask'].to(device)
            
            labels = batch['label'].to(device)

            outputs = model(input_ids, attention_mask = attention_mask, labels = labels)
            
            loss = outputs.loss
            
            total_loss += loss.item()

    return total_loss / len(data_loader)

In [11]:
# Método do loop de previsão
def predict(model, data_loader, device):
    
    model.eval()
    
    predictions = []

    with torch.no_grad():
        
        for batch in data_loader:
            
            input_ids = batch['input_ids'].to(device)
            
            attention_mask = batch['attention_mask'].to(device)

            outputs = model(input_ids, attention_mask = attention_mask)
            
            _, preds = torch.max(outputs.logits, dim = 1)
            
            predictions.extend(preds.tolist())

    return predictions

## Definição dos Dados

In [12]:
# Hiperparâmetros
EPOCHS = 10
BATCH_SIZE = 16
MAX_LENGTH = 64
LEARNING_RATE = 2e-5
RANDOM_SEED = 42

In [13]:
# Conjunto de dados de exemplo
texts = [
    'A velocidade da luz é aproximadamente 300.000 km/s.',
    'A Terra é plana e os répteis controlam o mundo.',
    'A fotossíntese é um processo importante para as plantas.',
    'As vacas podem voar e atravessar paredes de concreto.',
    'O oxigênio é essencial para a respiração dos seres vivos.',
    'Os cavalos podem falar como seres humanos.',
    'As crianças aprendem a partir dos exemplos dos pais.',
    'As palavras verdadeiras não são agradáveis e as agradáveis não são verdadeiras.',
    'Leopardos trabalham de terno e gravata em frente ao computador.',
    'Carros voadores estão por toda parte.'
]

In [14]:
labels = [0, 1, 0, 1, 0, 1, 0, 0, 1, 1]  # 0: normal, 1: anômala

In [15]:
# Divisão dos dados em treino e teste
train_texts, test_texts, train_labels, test_labels = train_test_split(texts, 
                                                                      labels, 
                                                                      test_size = 0.2, 
                                                                      random_state = RANDOM_SEED)

## Tokenização dos Dados e Criação dos DataLoaders

In [16]:
# Nome do modelo pré-treinado com 110M de parâmetros
PRETRAINED_MODEL = 'bert-base-uncased'

https://huggingface.co/bert-base-uncased

In [17]:
# Inicializa o tokenizador
tokenizer = BertTokenizer.from_pretrained(PRETRAINED_MODEL)

In [18]:
# Tokenização dos dados
train_dataset = TokenizaDados(train_texts, train_labels, tokenizer, MAX_LENGTH)
test_dataset = TokenizaDados(test_texts, test_labels, tokenizer, MAX_LENGTH)

In [19]:
# Data Loaders
train_loader = DataLoader(train_dataset, batch_size = BATCH_SIZE, shuffle = True)
test_loader = DataLoader(test_dataset, batch_size = BATCH_SIZE)

In [20]:
# Configuração do dispositivo
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

## Construção, Treinamento e Avaliação do Modelo

In [21]:
# Importa o modelo pré-treinado
modelo = BertForSequenceClassification.from_pretrained(PRETRAINED_MODEL, num_labels = 2)

In [22]:
# Coloca o modelo na memória do device
modelo.to(device)

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12,

In [23]:
# Configuração do otimizador e critério de perda
optimizer = torch.optim.AdamW(modelo.parameters(), lr = LEARNING_RATE)
criterion = torch.nn.CrossEntropyLoss()

In [24]:
# Treinamento e validação do modelo
for epoch in range(EPOCHS):
    
    train_loss = treina_modelo(modelo, train_loader, criterion, optimizer, device)
    
    test_loss = avalia_modelo(modelo, test_loader, criterion, device)
    
    print(f'Epoch {epoch+1}/{EPOCHS}, Train Loss: {train_loss}, Test Loss: {test_loss}')

Epoch 1/10, Train Loss: 1.0227949619293213, Test Loss: 0.5277878046035767
Epoch 2/10, Train Loss: 0.655776858329773, Test Loss: 1.0249435901641846
Epoch 3/10, Train Loss: 0.5951642394065857, Test Loss: 1.0504977703094482
Epoch 4/10, Train Loss: 0.5804691910743713, Test Loss: 1.0582799911499023
Epoch 5/10, Train Loss: 0.5210273861885071, Test Loss: 0.9537007808685303
Epoch 6/10, Train Loss: 0.46127769351005554, Test Loss: 0.884994387626648
Epoch 7/10, Train Loss: 0.36648836731910706, Test Loss: 0.76836097240448
Epoch 8/10, Train Loss: 0.3374147415161133, Test Loss: 0.615771472454071
Epoch 9/10, Train Loss: 0.28344979882240295, Test Loss: 0.48208242654800415
Epoch 10/10, Train Loss: 0.231851264834404, Test Loss: 0.3869282603263855


## Deploy e Uso do Modelo Treinado

In [25]:
# Teste de detecção de anomalias
novos_dados = ['A gravidade mantém os planetas em órbita ao redor do Sol.',
               'Os carros podem nadar no oceano como peixes.']

In [26]:
# Tokeniza a amostra de dados
novo_dataset = TokenizaDados(novos_dados, [0] * len(novos_dados), tokenizer, MAX_LENGTH)

In [27]:
# Cria o dataloader
novo_loader = DataLoader(novo_dataset, batch_size = BATCH_SIZE)

In [28]:
# Faz as previsões com o modelo
previsoes = predict(modelo, novo_loader, device)

In [29]:
for text, prediction in zip(novos_dados, previsoes):
    print(f'Sentença: {text} | Previsão: {"anômala" if prediction else "normal"}')

Sentença: A gravidade mantém os planetas em órbita ao redor do Sol. | Previsão: normal
Sentença: Os carros podem nadar no oceano como peixes. | Previsão: anômala


# Fim